In [35]:
import pandas as pd

In [5]:
## low count suppression function
def suppress(df_to_suppress,columns,n=10):
    for c in columns:
        if not c in df_to_suppress.columns:
            raise ValueError(f'column {c} not in dataframe')

    df = df_to_suppress
    for c in df.columns[df.columns.isin(columns)]:
        df[f'{c}'] = df[f'{c}'].astype(str)
        df[f'{c}'] = df[f'{c}'].apply(lambda x: '<10' if (int(x)>0 and int(x)<n ) else x)
    return df

In [6]:
def suppress_scalar(val):
    return '<10' if int(val) <10 else val

In [7]:
def print_remainder(df):
    print(f'Remaining people: {suppress_scalar(len(df.index))}')

In [8]:
df = pd.read_csv('../output/input_exclusioncounts.csv')

## Preliminary exclusions
Remove people who have died or are no longer registered at the practice at the time of study

In [ ]:
print(f'People having been recorded as having died: {suppress_scalar(len(df[df.has_died == 1].index))}')
df = df[df.has_died == 0]
print_remainder(df)

In [ ]:
print(f'People having been recorded as not being registered: {suppress_scalar(len(df[df.registered == 0].index))}')
df = df[df.registered == 1]
print_remainder(df)

## Sex
Include people recorded as having a sex of 'M' or 'F'

In [ ]:
df = df[(df.sex == 'M') | (df.sex == 'F')]
print_remainder(df)

## Age
Three cohorts included based on age at time of positive COVID-19 test and PRIMIS sheilding flags (estimated expected proportion):
* over 65s (85%)
* over 55s (15%)
  - with shielding flag (5%)
  - over 55s with non-shielding flag (10%)

In [33]:
df['age_band'] =  df.apply(lambda x: '65_plus' if x.age>=65 else '55_65' if (x.age>=55 and x.age<65) else 'lt_55',axis=1)
df['cohort'] = df.apply(lambda x: x.age_band if x.age_band == '65_plus' else x.age_band if x.age_band == 'lt_55' else str(x.age_band) + ('H' if x.primis_shield==1 else 'L' if x.primis_nonshield==1 else 'N'),axis=1)
df_cohort_counts = df.groupby('cohort').count()['patient_id'].reset_index().rename(columns={'patient_id':'Patient count',"cohort":"Cohort"})
suppressed_df_cohort_counts = suppress(df_cohort_counts,['Patient count'])
suppressed_df_cohort_counts.style.hide_index()

Cohort,Patient count
55_65H,12
55_65L,120
65_plus,191
lt_55,677


In [9]:
df=df[(df.age_band!='lt_55')&(df.age_band!='55_65N')]
print_remainder(df)

Remaining people: 1000


## PCR Testing
Only patients identified as having COVID-19 by PCR testing to be included in this study

In [34]:
print(f'Most recent positive test date: {pd.to_datetime(df.first_positive_test_date).max()}')

Most recent positive test date: 2021-07-29 00:00:00


In [30]:
excl_count = len(df[df.first_positive_test_type == "LFT_Only"].index)
print(f'Patients identified as COVID-positive only by lateral flow test {suppress_scalar(excl_count)}')
df = df[df.first_positive_test_type != "LFT_Only"]
print_remainder(df)

Patients identified as COVID-positive only by lateral flow test <10
Remaining people: 1000


# Hospital Admissions
Exclude persons identified as 
* having been admitted to hospital with a
  - diagnosis of COVID-19 or 
  - primary diagnosis of COVID-19
* having attended amergency care with a diagnosis of COVID-19

In [22]:
df_hosp = df.groupby(['primary_covid_hospital_admission','covid_admission','covid_emergency_admission']).count()['patient_id'].reset_index()
df_hosp.insert(0,'cohort','All')
df_hosp = pd.concat([df_hosp,df.groupby(['cohort','primary_covid_hospital_admission','covid_admission','covid_emergency_admission']).count()['patient_id'].reset_index()],ignore_index=True)
df_hosp.rename(columns={'patient_id':'Count of patients','primary_covid_hospital_admission':'Admitted w/primary CV19 diagnosis','covid_admission':'Admitted w/CV19 diagnosis','covid_emergency_admission':'Emergency Admission w/CV19 diagnosis'},inplace=True)
df_hosp = suppress(df_hosp,['Count of patients'])
df_hosp.style.hide_index()

cohort,Admitted w/primary CV19 diagnosis,Admitted w/CV19 diagnosis,Emergency Admission w/CV19 diagnosis,Count of patients
All,0,0,0,629
All,0,0,1,33
All,0,1,0,32
All,0,1,1,<10
All,1,0,0,277
All,1,0,1,11
All,1,1,0,12
55_65H,0,0,0,<10
55_65H,0,0,1,<10
55_65H,1,0,0,<10


# Previous Steroid Prescription
Exclude patients recorded as having recieved a prescription for inhaled or oral steroids in the three months (90 days) leading up the issuance of the guidance

In [29]:
df_prev_ccs = df.groupby('has_previous_steroid_prescription').count()['patient_id'].reset_index()
df_prev_ccs.insert(0,'cohort','All')
df_prev_ccs = pd.concat([df_prev_ccs,df.groupby(['cohort','has_previous_steroid_prescription']).count()['patient_id'].reset_index()],ignore_index=True)
df_prev_ccs.rename(columns={'patient_id':'Count of patients','has_previous_steroid_prescription':'Having previous corticosteroid prescription'},inplace=True)
df_prev_ccs = suppress(df_prev_ccs,['Count of patients'])
df_prev_ccs.style.hide_index()

cohort,Having previous corticosteroid prescription,Count of patients
All,0,500
All,1,500
55_65H,0,<10
55_65H,1,<10
55_65L,0,55
55_65L,1,65
65_plus,0,101
65_plus,1,90
lt_55H,0,33
lt_55H,1,39


# Hospitalisation x Previous Steroid Prescription
Cross-tabluation of previous two exclusion criteria to understand interactions

In [50]:
hospXccs = pd.pivot_table(df,values='patient_id',index=['primary_covid_hospital_admission','covid_admission','covid_emergency_admission'],columns='has_previous_steroid_prescription',aggfunc='count',fill_value=0,margins=True)
hospXccs.rename(columns=lambda y: f'{y}',inplace=True)
hospXccs = suppress(hospXccs,hospXccs.columns)
hospXccs

has_previous_steroid_prescription                                             0  \
primary_covid_hospital_admission covid_admission covid_emergency_admission        
0                                0               0                          319   
                                                 1                           14   
                                 1               0                           19   
                                                 1                          <10   
1                                0               0                          134   
                                                 1                          <10   
                                 1               0                          <10   
All                                                                         500   

has_previous_steroid_prescription                                             1  \
primary_covid_hospital_admission covid_admission covid_emergency_admission        
0                                0               0                          310   
                                                 1                           19   
                                 1               0                           13   
                                                 1                          <10   
1                                0               0                          143   
                                                 1                          <10   
                                 1               0                          <10   
All                                                                         500   

has_previous_steroid_prescription                                            All  
primary_covid_hospital_admission covid_admission covid_emergency_admission        
0                                0               0                           629  
                                                 1                            33  
                                 1               0                            32  
                                                 1                           <10  
1                                0               0                           277  
                                                 1                            11  
                                 1               0                            12  
All                                                                         1000

In [51]:
df=df[(df.primary_covid_hospital_admission==0)&(df.covid_admission==0)&(df.covid_emergency_admission==0)&(df.has_previous_steroid_prescription==0)]
print_remainder(df)

Remaining people: 319


## GP Consultation
The presence of a GP consultation within 7 days prior to, or 7 days post the first positive test result is included as an explanatory variable rather than an exclusion criteria due to uncertainty about its recording.

In [52]:
excl_count = len(df[df.with_consultation==0].index)
print(f'Patients without GP consultation +/- 7 days from positive test {suppress_scalar(excl_count)}')
df = df[df.with_consultation == 1]
print_remainder(df)

Patients without GP consultation +/- 7 days from positive test 166
Remaining people: 153
